### Installation

In [2]:
#!pip install unsloth #==2025.9.7 #transformers==4.56.1

In [2]:
#pip install unsloth

In [6]:
#!pip uninstall -y protobuf
#!pip install protobuf==3.20.3

# Importing

In [7]:
from datasets import load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig
import torch
from huggingface_hub import notebook_login,login
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [9]:
notebook_login()

### Unsloth

In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-1.7B",
    max_seq_length=5000,
    dtype=torch.float16,
    load_in_4bit=False,
    device_map={"": 0}
)

==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing = False,
)

Unsloth 2025.12.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep

In [12]:
from datasets import load_dataset
reasoning_dataset = load_dataset("moremilk/CoT_Rare-Diseases_And_Health-Conditions", split="train")

#reasoning_val = load_dataset("DSTI/traffic-accidents-reports-kd-smollm2-360M-7k", split="eval")

non_reasoning_dataset = load_dataset("mlabonne/FineTome-100k", split="train")

README.md:   0%|          | 0.00/17.2k [00:00<?, ?B/s]

CoT_Rare Disseases_Health Conditions_9.8(…):   0%|          | 0.00/67.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9800 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Let's see the structure of both datasets:

In [13]:
reasoning_dataset

Dataset({
    features: ['answer', 'id', 'metadata', 'question'],
    num_rows: 9800
})

In [14]:
non_reasoning_dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

We now convert the reasoning dataset into conversational format:

In [9]:
def generate_conversation(examples):
    problems   = examples["query"]
    reasoning  = examples["chain_of_thought"]
    answers    = examples["response"]

    conversations = []
    for problem, reason, answer in zip(problems, reasoning, answers):
        assistant_response = (
            f"<think>{reason}</think>\n\n"
            f"<answer>{answer}</answer>"
        )
        conversations.append([
            {"role": "user", "content": problem},
            {"role": "assistant", "content": assistant_response}
        ])
    return { "conversations": conversations }

In [15]:
#Special 

def generate_conversation(examples):
    problems = examples["question"]
    metadata = examples["metadata"]
    answers = examples["answer"]

    conversations = []
    for problem, meta, answer in zip(problems, metadata, answers):
        assistant_response = (
            f"<think>{meta.get('reasoning', '')}</think>\n\n"
            f"<answer>{answer}</answer>"
        )
        conversations.append([
            {"role": "user", "content": problem},
            {"role": "assistant", "content": assistant_response},
        ])
    return {"conversations": conversations}

In [11]:
def generate_conversation(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]

    conversations = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Construct the user prompt
        if input_text:
            question = f"{instruction}\n\n{input_text}"
        else:
            question = instruction

        # Build the conversation format
        conversations.append([
            {"role": "user", "content": question},
            {"role": "assistant", "content": output}
        ])

    return {"conversations": conversations}

In [9]:
def generate_conversation(examples):
    problems  = examples["question"]
    solutions = examples["answer"]

    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [12]:
import ast  # Güvenli şekilde string->list/dict dönüşümü için
import re

def generate_conversation(examples):
    conversations = []

    for chat in examples["messages"]:
        try:
            # Eğer chat formatı düzgün bir Python listesi gibi ise
            parsed = ast.literal_eval(chat)

            # Sadece role ve content içeren dict'leri al
            conversation = [
                {"role": item["role"], "content": item["content"]}
                for item in parsed
                if isinstance(item, dict) and "role" in item and "content" in item
            ]

            # Eğer hem user hem assistant varsa, ekle
            if any(msg["role"] == "user" for msg in conversation) and any(msg["role"] == "assistant" for msg in conversation):
                conversations.append(conversation)

        except (ValueError, SyntaxError):
            # Dönüştürülemeyen kayıtları atla
            continue

    return {"conversations": conversations}

In [16]:
reasoning_dataset = reasoning_dataset.map(generate_conversation, batched=True)

reasoning_conversation = [
    tokenizer.apply_chat_template(conv, tokenize=False)
    for conv in reasoning_dataset["conversations"]
]

Map:   0%|          | 0/9800 [00:00<?, ? examples/s]

In [17]:
#reasoning_conversation = tokenizer.apply_chat_template(
    #reasoning_dataset.map(generate_conversation, batched = True)["conversations"],
    #tokenize = False,
#)

#reasoning_conversation_val = tokenizer.apply_chat_template(
    #reasoning_val.map(generate_conversation, batched = True)["conversations"],
    #tokenize = False,
#)

Let's see the first transformed row:

In [18]:
reasoning_conversation[0]

'<|im_start|>user\nA rare genetic disorder is known to be caused by a recessive allele "r". If two carrier parents (both with genotype Rr) have a child, what is the probability that the child will have the disorder (rr), be a carrier (Rr), or be unaffected (RR)?  Explain the underlying genetic principles and show how to calculate the probabilities.<|im_end|>\n<|im_start|>assistant\n<think>\nTo tackle this effectively, my approach begins with understanding the inheritance pattern of recessive traits. First, I needed to recall that a recessive allele only manifests its trait when an individual has two copies of it (homozygous recessive – rr in this case).  A carrier has one copy of the recessive allele and one copy of the dominant allele (heterozygous – Rr). They don\'t exhibit the disorder but can pass on the recessive allele.\n\nInitially, I need to examine the parents\' genotypes: both are Rr (carriers). This leads me to constructing a Punnett Square, a visual tool used to predict the

Next we take the non reasoning dataset and convert it to conversational format as well.

We have to use Unsloth's `standardize_sharegpt` function to fix up the format of the dataset first.

In [19]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(non_reasoning_dataset)

non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/100000 [00:00<?, ? examples/s]

UndefinedError: 'list object' has no attribute 'content'

Let's see the first row

In [20]:
non_reasoning_conversations[0]

NameError: name 'non_reasoning_conversations' is not defined

Now let's see how long both datasets are:

In [21]:
print(len(reasoning_conversation))
print(len(non_reasoning_conversations))

9800


NameError: name 'non_reasoning_conversations' is not defined

The non reasoning dataset is much longer. Let's assume we want the model to retain some reasoning capabilities, but we specifically want a chat model.

Let's define a ratio of chat only data. The goal is to define some mixture of both sets of data.

Let's select 25% reasoning and 75% chat based:

Let's sample the reasoning dataset by 25% (or whatever is 100% - chat_percentage)

In [22]:
import pandas as pd

non_reasoning = pd.Series(non_reasoning_conversations)
#n = int(len(non_reasoning) * chat_percentage)
non_reasoning_subset = non_reasoning.sample(
    0,
    random_state=2407,
    ignore_index=True
)

NameError: name 'non_reasoning_conversations' is not defined

In [18]:
import pandas as pd

reasoning = pd.Series(reasoning_conversation)
reasoning_subset = reasoning.sample(
    3007,
    random_state=2407,
    ignore_index=True
)

Finally combine both datasets:

In [24]:
data = pd.concat([
    pd.Series(reasoning_conversation),
    #pd.Series(reasoning_subset)
    #pd.Series(non_reasoning_subset)
])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
#combined_dataset = combined_dataset.shuffle(seed = 3407)

In [18]:
data_val = pd.concat([
    pd.Series(reasoning_conversation_val),
    pd.Series(non_reasoning_subset)
])
data_val.name = "text"

from datasets import Dataset
combined_dataset_val = Dataset.from_pandas(pd.DataFrame(data_val))

<a name="Train"></a>
### Train the model


In [25]:
training_args = SFTConfig(
    output_dir="./Nizami-1.7B",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    learning_rate=3e-5,
    logging_steps=10,
    fp16=True,
    report_to="none",
    gradient_checkpointing = False,
    dataloader_num_workers=16,
    num_train_epochs=2,
    weight_decay=0.01,
    warmup_steps=61,
    lr_scheduler_type = "cosine",
    #eval_strategy="steps",         # or "epoch"
    #eval_steps=10,                # how frequently to run validation
    #save_strategy="epoch",         # optional: save model after each epoch
    #save_total_limit=2   
)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=combined_dataset,
    #eval_dataset=combined_dataset_val,  # ✅ Add this line
    dataset_text_field="text",
    max_seq_length=5000
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/9800 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [18]:
#torch.cuda.empty_cache()
#torch.cuda.ipc_collect()

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [26]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,800 | Num Epochs = 2 | Total steps = 1,226
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 17,432,576 of 1,738,007,552 (1.00% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.609000
20,1.604100
30,1.503400
40,1.389700
50,1.339100
60,1.286500
70,1.245600


KeyboardInterrupt: 

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [32]:
model.save_pretrained("./Qwen3")
tokenizer.save_pretrained("./Qwen3")

('./Qwen3/tokenizer_config.json',
 './Qwen3/special_tokens_map.json',
 './Qwen3/chat_template.jinja',
 './Qwen3/vocab.json',
 './Qwen3/merges.txt',
 './Qwen3/added_tokens.json',
 './Qwen3/tokenizer.json')

In [33]:
import shutil

shutil.make_archive("Qwen3", "zip", "Qwen3")
shutil.move("Qwen3.zip", "/kaggle/working/Qwen3")

'/kaggle/working/Qwen3/Qwen3.zip'

In [31]:
question="""
How is the D-dimer assay utilized in the context of suspected Venous Thromboembolism (VTE), and what are the clinical implications of a negative result, particularly concerning the Wells' score in clinical decision-making algorithms?
"""

messages = [
    {"role" : "user", "content" : question}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True,
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 2500,
    temperature = 0.6,
    top_p = 0.95,
    top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
First, I needed to define the Heisenberg Uncertainty Principle and its core idea, emphasizing the inherent uncertainty in the measurement of certain pairs of physical quantities. I started by recalling the mathematical expression of the uncertainty principle, which is ΔxΔp ≥ ħ/2, where Δx is the uncertainty in position, Δp is the uncertainty in momentum, and ħ is the reduced Planck constant. I then described the physical implications, including the limits on simultaneous measurement of position and momentum, and the role of quantum fluctuations in this context. Next, I addressed common misconceptions, such as the principle being about measurement accuracy or being universally applicable, to ensure a comprehensive understanding. I included a breakdown of the uncertainty principle in different forms (energy-time, position-momentum, etc.) to cover its broader applicability. I also provided examples to illustrate its practical implications and its relevance to quantum mechanics. Fi